In [5]:
"""

Aomar TEBIB, Mathieu HUGO, Ilies MOMTAZ

GitHub: https://github.com/uvsq22200607/Mastermind_TD03

"""
import tkinter as tk
import tkinter.messagebox
from random import randint
import copy


# Initialisation des paramètres
colors=["white","red","blue","green","yellow","purple","brown","black","orange","pink"] #liste des couleurs possibles
ligne = 0
chargé = False
tableau_count=[]
tableau_canvas = []
tableau_circle = []
tableau_label = []
tableau_answer = []

# LES FONCTIONS :

def sauvegarder():
    sauvegarde = open('save.txt', 'w')
    #savegarde nb_ligne
    sauvegarde.write(str(nb_ligne)+"\n")
    
    #sauvegarde nb_couleur
    sauvegarde.write(str(nb_couleur)+"\n")

    #sauvegarde taille_code
    sauvegarde.write(str(taille_code)+"\n")

    #sauvegarde ligne
    sauvegarde.write(str(ligne)+"\n")

    #sauvegarde la combinaison
    for i in range(taille_code):
        sauvegarde.write(str(combinaison[i]))
    sauvegarde.write("\n")
    
    #sauvegarde de tableau_count
    for i in range(nb_ligne):
        for j in range(taille_code):
            sauvegarde.write(str(tableau_count[i][j]))
    sauvegarde.write("\n")
    #sauvegarde les réponses
    save_answer = ""
    for i in range(len(tableau_answer)):
        save_answer += tableau_answer[i]
    sauvegarde.write(save_answer)
    sauvegarde.close()

def charger():
    global combinaison
    global ligne
    global nb_ligne
    global nb_couleur
    global taille_code
    global tableau_count
    global save
    global chargé
    chargé = True
    sauvegarde = open('save.txt', 'r')
    save = sauvegarde.readlines() #prend chaques lignes de ma sauvegarde et les mets dans une liste 
    #charge nb_ligne
    save_nb_ligne = list(save[0]) 
    nb_ligne = ""
    for i in range(len(save[0])-1):
        nb_ligne += save_nb_ligne[i]
    nb_ligne = int(nb_ligne)

    #charge nb_couleur
    save_nb_couleur = list(save[1])
    nb_couleur = ""
    for i in range(len(save[0])-1):
        nb_couleur += save_nb_couleur[i]
    nb_couleur = int(nb_couleur)

    #charge taille_code
    save_taille_code = list(save[2])
    taille_code = ""
    for i in range(len(save[0])-1):
        taille_code += save_taille_code[i]
    taille_code = int(taille_code)

    #charge ligne
    save_ligne = list(save[3])
    ligne = ""
    for i in range(len(save[0])-1):
        ligne += save_ligne[i]
    ligne = int(ligne)

    #charge la combinaison
    new_comb = []
    save_comb = list(save[4])
    for i in range(taille_code):
        save_comb[i] = int(save_comb[i])
        new_comb.append(save_comb[i])
    combinaison = new_comb

    #charge tableau_count
    new_count = []
    save_count = list(save[5])
    save_count.pop()
    for i in range(len(save_count)):
        save_count[i] = int(save_count[i])
    c=0
    for i in range(nb_ligne):
        row = []
        for j in range(taille_code):
            row.append(save_count[c])
            c+=1
        new_count.append(row)
    tableau_count=new_count

    selection.destroy()
    
def ctrl_z():
    global ligne
    for m in range(taille_code):
        tableau_count[ligne][m] = 0 #reinitialise les compteurs de la ligne ou l'on est
        tableau_canvas[ligne][m].itemconfig(circle, fill=colors[0]) #reinitialise les couleurs de la ligne ou l'on est
    ligne-= 1 
    tableau_label[ligne].config(text ="") #enlève le texte du label
    tableau_answer.pop() # enlève le dernier résultat qui a été ajouté

def quitter():
    global root2
    root2 =tk.Tk()
    label_root2 = tk.Label(root2, text="voulez-vous vraiment quitter ?")
    label_root2.grid(row=0, column=1)
    button1 = tk.Button(root2, text="oui", command= lambda : rep(0))
    button1.grid(row=1, column=0)
    button2 = tk.Button(root2, text="non", command= lambda : rep(1))
    button2.grid(row=1, column=2)
    root2.mainloop()

def rep(a):
    if a == 0:
        root2.destroy()
        root.destroy()
    if a == 1:
        root2.destroy()

    
def color(column):
    tableau_count[ligne][column] += 1 #augmente le compteur associé au canevas de la ligne et colonne concernées
    tableau_canvas[ligne][column].itemconfig(circle, fill=colors[tableau_count[ligne][column]%nb_couleur]) #change la couleur du canevas en foction de son compteur
    
def validate():
    global ligne
    global tableau_answer
    ganswer = 0
    banswer = 0
    combinaison_copy = combinaison.copy() #sans le .copy(), combinaison aurait aussi été modifié
    
    # vérifie les bonnes couleurs bien placées
    for i in range(len(combinaison)):
        if tableau_count[ligne][i]%nb_couleur+1 == combinaison[i]:
            ganswer+=1  
    
    # vérifie le nombre de couleurs proposé par le joueur qui sont dans la combinaison
    for i in range(len(combinaison)):
        if tableau_count[ligne][i]%nb_couleur+1 in combinaison_copy:
            combinaison_copy.remove(tableau_count[ligne][i]%nb_couleur+1)
            banswer+=1

    tableau_answer.append(str(ganswer)+str(banswer-ganswer))
    tableau_label[ligne].config(text = "bonnes couleurs, bien placées : " + str(ganswer)  + "\n" + "bonnes couleurs, mal placées : " + str(banswer-ganswer))
    ligne+=1 # on passe à la ligne suivante
    if ganswer == taille_code:
        tkinter.messagebox.showinfo("CONGRATULATIONS","Well played, you won!") # créer une fenêtre qui te dis que tu as gagné
        root.destroy()
    if ligne == nb_ligne and ganswer != taille_code:
        tkinter.messagebox.showinfo("YOU LOST","You'll do better next time") # créer une fenêtre qui te dis que tu as perdu
        root.destroy()


def start(joueur): # fonction qui sert à vérifier que le joueur a correctement remplis les cases, prendre les paramètre pour définir nos variables, et créer la combinaison
    global nb_couleur
    global nb_ligne
    global taille_code
    global tableau_entry
    global label_cb_error
    global cb
    global combinaison
    
    # .isnumeric() renvoie True ou False en fonction si c'est un nombre ou non
    color_check = (entree_couleur.get()).isnumeric() 
    ligne_check = (entree_ligne.get()).isnumeric()
    code_check = (entree_code.get()).isnumeric()

    if entree_couleur.get() == "" or entree_ligne.get() == "" or entree_code.get() == "":
        label_error.config(text="tu n'as pas remplie toutes les cases")
        return()
    
    if color_check == False or ligne_check == False or code_check == False:
        label_error.config(text="tu n'as pas bien remplie les cases")
        return

    nb_couleur = int(entree_couleur.get())
    nb_ligne = int(entree_ligne.get())
    taille_code = int(entree_code.get())
    
    if  nb_ligne > 10 or nb_ligne < 2:
        label_error.config(text="Le nombre de lignes doit être compris entre 2 et 10")
        return()

    if  nb_couleur>len(colors) or nb_couleur < 2:
        label_error.config(text="Le nombre de couleurs doit être compris entre 2 et " + str(len(colors)))
        return()
    
    if  taille_code>10 or taille_code < 2:
        label_error.config(text="La taille du code doit être compris entre 2 et 10")
        return()
    
    # création de la combinaison en fonction de si le joueur joue seul ou à 2
    if joueur == 1:
        combinaison = []
        for g in range(taille_code):
            combinaison.append(randint(1,nb_couleur))
    selection.destroy()
    if joueur == 2:
        cb=tk.Tk()
        tableau_entry=[]
        légende = ""
        for e in range(1,taille_code+1):
            entry_cb = tk.Entry(cb, width = 2, justify= "center")
            entry_cb.grid(row=1, column=e)
            tableau_entry.append(entry_cb)
        label_cb=tk.Label(cb, text="Rentrez une combinaison de " + str(taille_code) + " couleurs")
        label_cb.grid(row=1, column=0)
        for z in range(nb_couleur):
            légende+= str(colors[z]) + " = " + str(z+1) +"\n"
        label_légende=tk.Label(cb, text=légende)
        label_légende.grid(row=1, column=99)
        label_cb_error=tk.Label(cb, text="")
        label_cb_error.grid(row=2, column=0)
        bouton_cb = tk.Button(cb, text="valider", command=valider_cb)
        bouton_cb.grid(row=2, column=99)
        cb.mainloop()
    

def valider_cb():
    global combinaison
    combinaison = []
    for a in range(taille_code):
        if tableau_entry[a].get() == "" or int(tableau_entry[a].get())<1 or int(tableau_entry[a].get())> nb_couleur or (tableau_entry[a].get()).isnumeric() == False:
            label_cb_error.config(text="tu n'as pas correctement remplie toutes les cases")
            return()
        combinaison.append(int(tableau_entry[a].get()))
    cb.destroy()




# écran de sélection du nombre de lignes et de couleurs
selection = tk.Tk()
selection.title("SELECTION")

label_ligne = tk.Label(text="combien de ligne ? (2-10)")
label_ligne.grid(row=1, column=0)
entree_ligne = tk.Entry(selection, width = 5, justify= "center")
entree_ligne.grid(row = 1, column = 1)

label_couleur = tk.Label(text="combien de couleurs ? (2-"+ str(len(colors)) + ")")
label_couleur.grid(row=2, column=0)
entree_couleur = tk.Entry(selection, width = 5, justify= "center")
entree_couleur.grid(row = 2, column = 1)

label_code = tk.Label(text="taille du code secret (2-10)")
label_code.grid(row=3, column=0)
entree_code = tk.Entry(selection, width = 5, justify= "center")
entree_code.grid(row = 3, column = 1)

label_error=tk.Label(selection, text="")
label_error.grid(row = 4, column = 2)

boutonj1=tk.Button(selection, text="1 joueur", command=lambda : start(1))
boutonj1.grid(row=4, column=0)
boutonj2=tk.Button(selection, text="2 joueur", command=lambda : start(2))
boutonj2.grid(row=4, column=1)

buttonCharger = tk.Button(selection, text="charger",command=charger)
buttonCharger.grid(row = 5, column= 0)

selection.mainloop()




# écran du jeu
root = tk.Tk()
root.title("MASTERMIND")

# création des canevas et des tableaux

# tableau de compteur
if tableau_count == []:
    for i in range(nb_ligne):
        row=[]
        for f in range(taille_code):
            row.append(0)
        tableau_count.append(row)

# tableau de canevas
for j in range(1,nb_ligne+1):
    ligne_boucle = []
    for i in range(1,taille_code+1):
        canvas = tk.Canvas(root, width=75, height= 75)
        canvas.grid(row=j, column= i)
        ligne_boucle.append(canvas)
    tableau_canvas.append(ligne_boucle)

# créer mes cercles et les ajoutent à mon tableau de cercle
for j in range(nb_ligne):
    ligne_boucle = []
    for i in range(taille_code):
        circle = tableau_canvas[j][i].create_oval(5, 5, 70, 70, fill=colors[tableau_count[j][i]%nb_couleur])
        ligne_boucle.append(circle)
    tableau_circle.append(ligne_boucle)

# tableau de label
for i in range(1,nb_ligne+1):
    lab_ga=tk.Label(root,text="")
    lab_ga.grid(column=0,row=i)
    tableau_label.append(lab_ga)

# créer les boutons
for b in range(taille_code):
    buttonColor = tk.Button(root, width=10, height=5, command=lambda col = b: color(col))
    buttonColor.grid(row = 99, column= b+1)                         #col = b est essentiel, sinon chaque bouton aurait comme commande la foction color(taille_code)

#charge les labels réponses, doit se faire dans ma fenêtre et pas dans ma foction charger car les labels ne sont créés qu'après avoir appelé ma foction
if chargé == True: # s'effectue uniquement si la fonction charger a été appelé
    for i in range(len(save[6])//2):
        tableau_label[i].config(text = "bonnes couleurs, bien placées : " + save[6][2*i]  + "\n" + "bonnes couleurs, mal placées : " + save[6][2*i+1])
        tableau_answer.append(str(save[6][2*i])+str(save[6][2*i+1]))


buttonQ = tk.Button(root, width=10, height=5, command=quitter,text="Quitter")
buttonQ.grid(row = 99, column= 0)

buttonCTRL_Z = tk.Button(root, width=10, height=5, command=ctrl_z,text="retour arrière")
buttonCTRL_Z.grid(row = 99, column= 97)

buttonV = tk.Button(root, width=10, height=5, text="Valider",command=validate)
buttonV.grid(row = 99, column= 99)

buttonS = tk.Button(root, width=10, height=5, text="sauvegarder",command=sauvegarder)
buttonS.grid(row = 99, column= 100)

root.mainloop()